>作者:刘早起
>
>公众号: 早起Python
<br><br>

# 前言
在Pandas中每一个DataFrame都有一个Style属性，我们可以通过修改该属性来给数据添加一些基本的样式。
<br><br>

# 使用说明
通过修改Styler对象的属性，将样式传递给DataFrame，主要有两种传递方式

- **Styler.applymap**：对所有的单元格操作样式
- **Styler.apply**：对列/行/表操作样式
<br><br>
Styler.applymap通过DataFrame对所有的单元格操作样式。
<br>
Styler.apply根据axis参数，按列使用axis=0，按行使用axis=1，以及axis=None作用于整个表。所以<br><br>
- 若使用Styler.applymap，我们的函数应返回带有**CSS属性-值对**的单个字符串。
- 若使用Styler.apply，我们的函数应返回具有相同形状的Series或DataFrame，其中每个值都是具有CSS属性值对的字符串。
<br><br>
不会CSS？没关系，作为调包侠的我们大多是改改HTML颜色代码即可完成样式修改，下面看一些示例。
<br><br>

# 一些例子
### 基本样式

首先我们创建一组没有任何样式的数据

In [55]:
import pandas as pd

df = pd.read_csv('test.csv')
df

,A,B,C,D,E
0,1.0,1.329212,NaN,-0.316280,-0.990810
1,2.0,-1.070816,-1.438713,0.564417,0.295722
2,3.0,-1.626404,0.219565,0.678805,1.889273
3,4.0,0.961538,0.104011,NaN,0.850229
4,5.0,1.453425,1.057737,0.165562,0.515018
5,6.0,-1.336936,0.562861,1.392855,-0.063328
6,7.0,0.121668,1.207603,-0.002040,1.627796
7,8.0,0.354493,1.037528,-0.385684,0.519818
8,9.0,1.686583,-1.325963,1.428984,-2.089354
9,10.0,-0.129820,0.631523,-0.586538,0.290720


我们之前说过，DataFrame是有style属性的，所以在没有做任何修改的情况下，使用df.style应该和上图一样


In [5]:
df.style

,A,B,C,D,E
0,1.000000,1.329212,nan,-0.316280,-0.990810
1,2.000000,-1.070816,-1.438713,0.564417,0.295722
2,3.000000,-1.626404,0.219565,0.678805,1.889273
3,4.000000,0.961538,0.104011,nan,0.850229
4,5.000000,1.453425,1.057737,0.165562,0.515018
5,6.000000,-1.336936,0.562861,1.392855,-0.063328
6,7.000000,0.121668,1.207603,-0.002040,1.627796
7,8.000000,0.354493,1.037528,-0.385684,0.519818
8,9.000000,1.686583,-1.325963,1.428984,-2.089354
9,10.000000,-0.129820,0.631523,-0.586538,0.290720


现在让我们编写一个简单的样式函数，该函数可以将负数变为红色，使正数保持黑色。

In [6]:
def color_negative_red(val):
    color = 'red' if val< 0 else 'black'
    return 'color: {}'.format(color)

现在来应用这段函数

In [7]:
df.style.applymap(color_negative_red)

,A,B,C,D,E
0,1.000000,1.329212,nan,-0.316280,-0.990810
1,2.000000,-1.070816,-1.438713,0.564417,0.295722
2,3.000000,-1.626404,0.219565,0.678805,1.889273
3,4.000000,0.961538,0.104011,nan,0.850229
4,5.000000,1.453425,1.057737,0.165562,0.515018
5,6.000000,-1.336936,0.562861,1.392855,-0.063328
6,7.000000,0.121668,1.207603,-0.002040,1.627796
7,8.000000,0.354493,1.037528,-0.385684,0.519818
8,9.000000,1.686583,-1.325963,1.428984,-2.089354
9,10.000000,-0.129820,0.631523,-0.586538,0.290720


现在如果我们想突出显示**每列**中的最大值，需要重新定义一个函数。

In [8]:
def highlight_max(series):
    is_max = series==series.max()
    return ['background-color: yellow' if v else '' 
            for v in is_max]

#测试一下
highlight_max(df['E'])

['', '', 'background-color: yellow', '', '', '', '', '', '', '']

之前我们是以单元格为单位进行操作，使用的是.applymap。现在我们待操作的对象是列，所以这里用的是.apply方法

In [9]:
df.style.apply(highlight_max)

,A,B,C,D,E
0,1.000000,1.329212,nan,-0.316280,-0.990810
1,2.000000,-1.070816,-1.438713,0.564417,0.295722
2,3.000000,-1.626404,0.219565,0.678805,1.889273
3,4.000000,0.961538,0.104011,nan,0.850229
4,5.000000,1.453425,1.057737,0.165562,0.515018
5,6.000000,-1.336936,0.562861,1.392855,-0.063328
6,7.000000,0.121668,1.207603,-0.002040,1.627796
7,8.000000,0.354493,1.037528,-0.385684,0.519818
8,9.000000,1.686583,-1.325963,1.428984,-2.089354
9,10.000000,-0.129820,0.631523,-0.586538,0.290720


现在可以使用

In [10]:
df.style.applymap(color_negative_red).apply(highlight_max)

,A,B,C,D,E
0,1.000000,1.329212,nan,-0.316280,-0.990810
1,2.000000,-1.070816,-1.438713,0.564417,0.295722
2,3.000000,-1.626404,0.219565,0.678805,1.889273
3,4.000000,0.961538,0.104011,nan,0.850229
4,5.000000,1.453425,1.057737,0.165562,0.515018
5,6.000000,-1.336936,0.562861,1.392855,-0.063328
6,7.000000,0.121668,1.207603,-0.002040,1.627796
7,8.000000,0.354493,1.037528,-0.385684,0.519818
8,9.000000,1.686583,-1.325963,1.428984,-2.089354
9,10.000000,-0.129820,0.631523,-0.586538,0.290720


当然我们也可以通过修改样式函数并使用.apply来高亮整个DataFrame的最大值，

In [16]:
import numpy as np

def highlight_max(data, color='yellow'):
    attr = 'background-color: {}'.format(color)
    if data.ndim ==1:
        is_max = data==data.max()
        return [attr if v else '' for v in is_max]
    else:
        is_max = data==data.max().max()
        return pd.DataFrame(np.where(is_max, attr, ''),
                           index=data.index, 
                           columns=data.columns)
    
df.style.apply(highlight_max, 
               color='darkorange', 
               axis=None)

,A,B,C,D,E
0,1.000000,1.329212,nan,-0.316280,-0.990810
1,2.000000,-1.070816,-1.438713,0.564417,0.295722
2,3.000000,-1.626404,0.219565,0.678805,1.889273
3,4.000000,0.961538,0.104011,nan,0.850229
4,5.000000,1.453425,1.057737,0.165562,0.515018
5,6.000000,-1.336936,0.562861,1.392855,-0.063328
6,7.000000,0.121668,1.207603,-0.002040,1.627796
7,8.000000,0.354493,1.037528,-0.385684,0.519818
8,9.000000,1.686583,-1.325963,1.428984,-2.089354
9,10.000000,-0.129820,0.631523,-0.586538,0.290720


### 选择某列
我们也可以通过**subset**对选中的某列或某几列进行样式修饰

In [18]:
df.style.apply(highlight_max, 
               axis=0, #列方向
               subset=['B', 'C', 'D']) #选择BCD列

,A,B,C,D,E
0,1.000000,1.329212,nan,-0.316280,-0.990810
1,2.000000,-1.070816,-1.438713,0.564417,0.295722
2,3.000000,-1.626404,0.219565,0.678805,1.889273
3,4.000000,0.961538,0.104011,nan,0.850229
4,5.000000,1.453425,1.057737,0.165562,0.515018
5,6.000000,-1.336936,0.562861,1.392855,-0.063328
6,7.000000,0.121668,1.207603,-0.002040,1.627796
7,8.000000,0.354493,1.037528,-0.385684,0.519818
8,9.000000,1.686583,-1.325963,1.428984,-2.089354
9,10.000000,-0.129820,0.631523,-0.586538,0.290720


### 格式化输出

我们也可以使用Styler.format来快速格式化输出，比如将小数格式化为百分数

In [19]:
df.style.format("{:.2%}")

,A,B,C,D,E
0,100.00%,132.92%,nan%,-31.63%,-99.08%
1,200.00%,-107.08%,-143.87%,56.44%,29.57%
2,300.00%,-162.64%,21.96%,67.88%,188.93%
3,400.00%,96.15%,10.40%,nan%,85.02%
4,500.00%,145.34%,105.77%,16.56%,51.50%
5,600.00%,-133.69%,56.29%,139.29%,-6.33%
6,700.00%,12.17%,120.76%,-0.20%,162.78%
7,800.00%,35.45%,103.75%,-38.57%,51.98%
8,900.00%,168.66%,-132.60%,142.90%,-208.94%
9,1000.00%,-12.98%,63.15%,-58.65%,29.07%


也支持使用字典或lambda表达式来更灵活的使用

In [40]:
df.style.format({'B': "{:.3f}", #保留小数点后3位
                'D': '{:+.3f}'})#带符号，保留小数点后三位

,A,B,C,D,E
0,1.000000,1.329,nan,-0.316,-0.990810
1,2.000000,-1.071,-1.438713,+0.564,0.295722
2,3.000000,-1.626,0.219565,+0.679,1.889273
3,4.000000,0.962,0.104011,+nan,0.850229
4,5.000000,1.453,1.057737,+0.166,0.515018
5,6.000000,-1.337,0.562861,+1.393,-0.063328
6,7.000000,0.122,1.207603,-0.002,1.627796
7,8.000000,0.354,1.037528,-0.386,0.519818
8,9.000000,1.687,-1.325963,+1.429,-2.089354
9,10.000000,-0.130,0.631523,-0.587,0.290720


### style内置方法
|方法|功能|
|:---|:---|
|style.format() |格式化|
|style.hide_columns(subset) |隐藏某列|
|style.hide_index()|隐藏行索引|
|style.background_gradient()|梯度下降着色|
|style.bar()|每个数值型单元格显示横向条形图|
|style.highlight_max(color='yellow') |df中每列最大的值着色|
|style.highlight_min(color='yellow')|df中每列最小的值着色|
|style.highlight_null(null_color='red')|df中缺失值着色|
|style.to_excel(xlsxfile) |将dataframe带着样式存到xlsx文件中|

In [78]:
#隐藏A列
df.style.hide_columns('A')

,B,C,D,E
0,1.329212,nan,-0.316280,-0.990810
1,-1.070816,-1.438713,0.564417,0.295722
2,-1.626404,0.219565,0.678805,1.889273
3,0.961538,0.104011,nan,0.850229
4,1.453425,1.057737,0.165562,0.515018
5,-1.336936,0.562861,1.392855,-0.063328
6,0.121668,1.207603,-0.002040,1.627796
7,0.354493,1.037528,-0.385684,0.519818
8,1.686583,-1.325963,1.428984,-2.089354
9,-0.129820,0.631523,-0.586538,0.290720


In [79]:
#隐藏行索引
df.style.hide_index()

A,B,C,D,E
1.000000,1.329212,nan,-0.316280,-0.990810
2.000000,-1.070816,-1.438713,0.564417,0.295722
3.000000,-1.626404,0.219565,0.678805,1.889273
4.000000,0.961538,0.104011,nan,0.850229
5.000000,1.453425,1.057737,0.165562,0.515018
6.000000,-1.336936,0.562861,1.392855,-0.063328
7.000000,0.121668,1.207603,-0.002040,1.627796
8.000000,0.354493,1.037528,-0.385684,0.519818
9.000000,1.686583,-1.325963,1.428984,-2.089354
10.000000,-0.129820,0.631523,-0.586538,0.290720


In [84]:
#梯度下降着色
df.style.background_gradient()

,A,B,C,D,E
0,1.000000,1.329212,nan,-0.316280,-0.990810
1,2.000000,-1.070816,-1.438713,0.564417,0.295722
2,3.000000,-1.626404,0.219565,0.678805,1.889273
3,4.000000,0.961538,0.104011,nan,0.850229
4,5.000000,1.453425,1.057737,0.165562,0.515018
5,6.000000,-1.336936,0.562861,1.392855,-0.063328
6,7.000000,0.121668,1.207603,-0.002040,1.627796
7,8.000000,0.354493,1.037528,-0.385684,0.519818
8,9.000000,1.686583,-1.325963,1.428984,-2.089354
9,10.000000,-0.129820,0.631523,-0.586538,0.290720


In [72]:
# 最大值着色
#df.style.highlight_max()

#最小值着色
df.style.highlight_min()

,A,B,C,D,E
0,1.000000,1.329212,nan,-0.316280,-0.990810
1,2.000000,-1.070816,-1.438713,0.564417,0.295722
2,3.000000,-1.626404,0.219565,0.678805,1.889273
3,4.000000,0.961538,0.104011,nan,0.850229
4,5.000000,1.453425,1.057737,0.165562,0.515018
5,6.000000,-1.336936,0.562861,1.392855,-0.063328
6,7.000000,0.121668,1.207603,-0.002040,1.627796
7,8.000000,0.354493,1.037528,-0.385684,0.519818
8,9.000000,1.686583,-1.325963,1.428984,-2.089354
9,10.000000,-0.129820,0.631523,-0.586538,0.290720


In [85]:
#保存样式到xlsx文件中
new_df = df.style.background_gradient()
new_df.to_excel('test.xlsx')

# 支持一下
![](img/my_zanshang_qrcode.jpg)